In [1]:
from keras.models import Sequential
#"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
from keras.layers import Input
from keras.models import Model

In [3]:
df=pd.read_csv("affwildtrainNew2.csv")
#df2=pd.read_csv("validation set AU.csv")
df.shape
df.head()

,S,ID,AU1_0,AU1_1,AU2_0,AU2_1,AU4_0,AU4_1,AU6_0,AU6_1,AU12_0,AU12_1,AU15_0,AU15_1,AU20_0,AU20_1,AU25_0,AU25_1
0,0,00001.jpg,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
1,1,00002.jpg,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
2,2,00003.jpg,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
3,3,00004.jpg,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
4,4,00005.jpg,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0


In [15]:
columns=['AU1_0','AU1_1','AU2_0','AU2_1','AU4_0','AU4_1','AU6_0','AU6_1','AU12_0','AU12_1','AU15_0','AU15_1','AU20_0','AU20_1','AU25_0','AU25_1']
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(dataframe=df[:300],directory="AW2Train",x_col="ID",y_col=columns,batch_size=32,seed=42,shuffle=True,class_mode="other",target_size=(100,100))
valid_generator=test_datagen.flow_from_dataframe(dataframe=df[300:400],directory="AW2Train",x_col="ID",y_col=columns,batch_size=32,seed=42,shuffle=True,class_mode="other",target_size=(100,100))
#test_generator=test_datagen.flow_from_dataframe(dataframe=df2[29000:40000],directory="validation set AU",x_col="ID",batch_size=1,seed=42,shuffle=False,class_mode=None,target_size=(100,100))
#train_generator[0]

Found 300 validated image filenames.
Found 100 validated image filenames.


In [5]:
from keras.callbacks import Callback
from keras.backend import clear_session
from keras.models import Model, load_model
from keras.layers import Dense, Input, Flatten
from keras.applications import ResNet50

In [6]:
img = Input(shape = (100, 100, 3))
model = ResNet50(include_top=False, weights='imagenet', input_tensor=img, input_shape=(100,100,3), pooling='avg')
x = model.layers[-1].output


output1 = Dense(2, activation = 'softmax',name='AU1')(x)
output2 = Dense(2, activation = 'softmax',name='AU2')(x)
output3 = Dense(2, activation = 'softmax',name='AU4')(x)
output4 = Dense(2, activation = 'softmax',name='AU6')(x)
output5 = Dense(2, activation = 'softmax',name='AU12')(x)
output6 = Dense(2, activation = 'softmax',name='AU15')(x)
output7 = Dense(2, activation = 'softmax',name='AU20')(x)
output8 = Dense(2, activation = 'softmax',name='AU25')(x)

model = Model(img,[output1,output2,output3,output4,output5,output6,output7,output8])
model.compile(optimizers.rmsprop(lr = 0.0001, decay = 1e-6),
loss = ["categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy"],metrics = ["accuracy"])
   

In [7]:
model.summary()
output1.shape

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 50, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 50, 50, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

TensorShape([Dimension(None), Dimension(2)])

In [8]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,0:2],batch_y[:,2:4],batch_y[:,4:6],batch_y[:,6:8],batch_y[:,8:10],batch_y[:,10:12],batch_y[:,12:14],batch_y[:,14:16]])
        #yield (batch_x,[batch_y[:,0],batch_y[:,1],batch_y[:,2],batch_y[:,3].batch_y[:,4],batch_y[:,5],batch_y[:,6],batch_y[:,7]])

In [16]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n
model.fit_generator(generator=generator_wrapper(train_generator),steps_per_epoch=STEP_SIZE_TRAIN,validation_data=generator_wrapper(valid_generator),validation_steps=STEP_SIZE_VALID,epochs=3,verbose=1)


Epoch 1/3
9/9 [==============================] - 41s 5s/step - loss: 0.8855 - AU1_loss: 0.1645 - AU2_loss: 0.0351 - AU4_loss: 0.0545 - AU6_loss: 0.0434 - AU12_loss: 0.5195 - AU15_loss: 0.0264 - AU20_loss: 0.0225 - AU25_loss: 0.0197 - AU1_acc: 0.9653 - AU2_acc: 1.0000 - AU4_acc: 1.0000 - AU6_acc: 1.0000 - AU12_acc: 0.8993 - AU15_acc: 1.0000 - AU20_acc: 1.0000 - AU25_acc: 1.0000 - val_loss: 3.6380 - val_AU1_loss: 2.5188 - val_AU2_loss: 6.5481e-05 - val_AU4_loss: 2.2601e-04 - val_AU6_loss: 6.4224e-05 - val_AU12_loss: 1.1187 - val_AU15_loss: 2.8966e-05 - val_AU20_loss: 2.5485e-05 - val_AU25_loss: 4.8294e-05 - val_AU1_acc: 0.8438 - val_AU2_acc: 1.0000 - val_AU4_acc: 1.0000 - val_AU6_acc: 1.0000 - val_AU12_acc: 0.7500 - val_AU15_acc: 1.0000 - val_AU20_acc: 1.0000 - val_AU25_acc: 1.0000
Epoch 2/3
9/9 [==============================] - 37s 4s/step - loss: 1.0573 - AU1_loss: 0.5123 - AU2_loss: 0.0080 - AU4_loss: 0.0135 - AU6_loss: 0.0116 - AU12_loss: 0.4972 - AU15_loss: 0.0054 - AU20_loss: 0.00

In [34]:
from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

#img = img.reshape(1, img_width, img_height, 3)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.02)

#we are calculating metrics for whole of train dataset
df2=pd.read_csv("validation set AU.csv")
y_test= df2.drop(['ID'], axis = 1)
y_test = y_test.to_numpy()
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(dataframe=df2[:40],directory="validation set AU",x_col="ID",batch_size=1,seed=42,shuffle=False,class_mode=None,target_size=(100,100))
STEP_SIZE_TEST=test_generator.n

y_pred=model.predict_generator(test_generator,steps=STEP_SIZE_TEST,verbose=1)

Found 40 validated image filenames.
40/40 [==============================] - 2s 58ms/step


In [35]:
num=test_generator.n
for i in range(num):
    for j in range(2):
        if(y_pred[0][i][j]>0.5):
            y_pred[0][i][j]=1
        else:
            y_pred[0][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[1][i][j]>0.5):
            y_pred[1][i][j]=1
        else:
            y_pred[1][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[2][i][j]>0.5):
            y_pred[2][i][j]=1
        else:
            y_pred[2][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[3][i][j]>0.5):
            y_pred[3][i][j]=1
        else:
            y_pred[3][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[4][i][j]>0.5):
            y_pred[4][i][j]=1
        else:
            y_pred[4][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[5][i][j]>0.5):
            y_pred[5][i][j]=1
        else:
            y_pred[5][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[6][i][j]>0.5):
            y_pred[6][i][j]=1
        else:
            y_pred[6][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[7][i][j]>0.5):
            y_pred[7][i][j]=1
        else:
            y_pred[7][i][j]=0


In [40]:
num=test_generator.n
rows, cols = (num, 8) 
arr2 = [[int(0)]*cols]*rows
arr2=np.array(arr2)
for i in range(num):
    if(y_pred[0][i][0].astype(int)==1):
        arr2[i][0]=0
    else:
        arr2[i][0]=1
for i in range(num):
    if(y_pred[1][i][0].astype(int)==1):
        arr2[i][1]=0
    else:
        arr2[i][1]=1
for i in range(num):
    if(y_pred[2][i][0].astype(int)==1):
        arr2[i][2]=0
    else:
        arr2[i][2]=1
for i in range(num):
    if(y_pred[3][i][0].astype(int)==1):
        arr2[i][3]=0
    else:
        arr2[i][3]=1
for i in range(num):
    if(y_pred[4][i][0].astype(int)==1):
        arr2[i][4]=0
    else:
        arr2[i][4]=1
for i in range(num):
    if(y_pred[5][i][0].astype(int)==1):
        arr2[i][5]=0
    else:
        arr2[i][5]=1
for i in range(num):
    if(y_pred[6][i][0].astype(int)==1):
        arr2[i][6]=0
    else:
        arr2[i][6]=1
for i in range(num):
    if(y_pred[7][i][0].astype(int)==1):
        arr2[i][7]=0
    else:
        arr2[i][7]=1

In [ ]:
y_pred=arr2

In [ ]:
print("Metrics for AU1\n")
y_test_AU1 = [0.0]*num
y_pred_AU1=[0.0]*num
for i in range(num):
    y_test_AU1[i]=y_test[i][0]
    y_pred_AU1[i]=y_pred[i][0]
print("\nTrue values\n")
print(y_test_AU1)
print("\nPredicted values\n")
print(y_pred_AU1)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU1, y_pred_AU1))
print(metrics.classification_report(y_test_AU1, y_pred_AU1))

In [ ]:
print("Metrics for AU2\n")
y_test_AU2 = [0.0]*num
y_pred_AU2=[0.0]*num
for i in range(num):
    y_test_AU2[i]=y_test[i][1]
    y_pred_AU2[i]=y_pred[i][1]
print("\nTrue values\n")
print(y_test_AU2)
print("\nPredicted values\n")
print(y_pred_AU2)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU2, y_pred_AU2))
print(metrics.classification_report(y_test_AU2, y_pred_AU2))

In [ ]:
print("Metrics for AU4\n")
y_test_AU4 = [0.0]*num
y_pred_AU4=[0.0]*num
for i in range(num):
    y_test_AU4[i]=y_test[i][2]
    y_pred_AU4[i]=y_pred[i][2]
print("\nTrue values\n")
print(y_test_AU4)
print("\nPredicted values\n")
print(y_pred_AU4)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU4, y_pred_AU4))
print(metrics.classification_report(y_test_AU4, y_pred_AU4))

In [ ]:
print("Metrics for AU6\n")
y_test_AU6 = [0.0]*num
y_pred_AU6=[0.0]*num
for i in range(num):
    y_test_AU6[i]=y_test[i][3]
    y_pred_AU6[i]=y_pred[i][3]
print("\nTrue values\n")
print(y_test_AU6)
print("\nPredicted values\n")
print(y_pred_AU6)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU6, y_pred_AU6))
print(metrics.classification_report(y_test_AU6, y_pred_AU6))

In [ ]:
print("Metrics for AU12\n")
y_test_AU12 = [0.0]*num
y_pred_AU12=[0.0]*num
for i in range(num):
    y_test_AU12[i]=y_test[i][4]
    y_pred_AU12[i]=y_pred[i][4]
print("\nTrue values\n")
print(y_test_AU12)
print("\nPredicted values\n")
print(y_pred_AU12)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU12, y_pred_AU12))
print(metrics.classification_report(y_test_AU12, y_pred_AU12))

In [ ]:
print("Metrics for AU15\n")
y_test_AU15 = [0.0]*num
y_pred_AU15=[0.0]*num
for i in range(num):
    y_test_AU15[i]=y_test[i][5]
    y_pred_AU15[i]=y_pred[i][5]
print("\nTrue values\n")
print(y_test_AU15)
print("\nPredicted values\n")
print(y_pred_AU15)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU15, y_pred_AU15))
print(metrics.classification_report(y_test_AU15, y_pred_AU15))

In [ ]:
print("Metrics for AU20\n")
y_test_AU20 = [0.0]*num
y_pred_AU20=[0.0]*num
for i in range(num):
    y_test_AU20[i]=y_test[i][6]
    y_pred_AU20[i]=y_pred[i][6]
print("\nTrue values\n")
print(y_test_AU20)
print("\nPredicted values\n")
print(y_pred_AU20)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU20, y_pred_AU20))
print(metrics.classification_report(y_test_AU20, y_pred_AU20))

In [ ]:
print("Metrics for AU25\n")
y_test_AU25 = [0.0]*num
y_pred_AU25=[0.0]*num
for i in range(num):
    y_test_AU25[i]=y_test[i][7]
    y_pred_AU25[i]=y_pred[i][7]
print("\nTrue values\n")
print(y_test_AU25)
print("\nPredicted values\n")
print(y_pred_AU25)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU25, y_pred_AU25))
print(metrics.classification_report(y_test_AU25, y_pred_AU25))

In [ ]:
columns=["AU1", "AU2", "AU4", "AU6", "AU12","AU15","AU20","AU25"]
#columns should be the same order of y_col
results=pd.DataFrame(y_pred, columns=columns)
results["Filenames"]=test_generator.filenames
ordered_cols=["Filenames"]+columns
results=results[ordered_cols]#To get the same column order
results.to_csv("grouping met 37k train, 11k test.csv",index=False)